In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from morse import Morse


# Atom masses in atomic mass units
mH, mF = 1., 19.

# Equilibrium bondlength in meters
re = 0.915801e-10

# dipole moment of HF at the equilibrium bondlength
dipole_au = -0.8113388


# potential energy at minimum, ok to set at zero
Te = 0

# Morse parameters in atomic units 
we_au, wexe_au = 0.01911889333688861, 0.0004944018798851136
# Morse parameters in wavenumbers
we, wexe = 4196.112054123941, 108.50866999528301

# create Morse object 
X = Morse(mH, mF, we, wexe, re, Te, dipole_au, 0.001)

# create Morse grid - will make grids in SI (self.r) and atomic units (self.r_au)
X.make_rgrid()

# create Morse potential on SI grid, will make potential in SI (self.V) and atomic units (self.V_au)
X.V = X.Vmorse(X.r)

print(F'Effective charge of HF is {X.q_au} atomic units')


# plot Morse potential in atomic units
plt.plot(X.r_au, X.V_au)
plt.xlabel("Bond length (atomic units)")
plt.ylabel("Potential (atomic units)")
plt.show()

In [ ]:
# calculate an eigenstate of the Morse oscillator
# self.psi_si will be normalized in SI units 
# self.psi_au will be normalized in atomic units

# calculate psi_20
psi = X.calc_psi_z(3)

# check normalization in SI units
norm_si = np.trapz(X.psi_si ** 2, X.r)

# check normalization in atomic units
norm_au = np.trapz(X.psi_au ** 2,  X.r_au)
print(F'Norm in au is {norm_au}')
print(F'Norm in si is {norm_si}')

# plot the normalized eigenfunction in atomic units against the potential in atomic units
plt.plot(X.r_au, X.V_au)
plt.plot(X.r_au, X.psi_au)
#plt.plot(X.r_au, X.r_au-X.r_eq_au)
plt.show()

In [ ]:
print(X.compute_Morse_transition_au(0, 1))
print(X.compute_Morse_transition_wn(0, 1))

print(X.compute_coupling_element_p_dot_A(0, 1, 1, 2))
x_01 = X.position_matrix_element(0,1)
deltaE_01 = X.compute_Morse_transition_au(1, 0)


print(F'charge: {X.q_au}')
print(F'A0: {X.A0_au}')
print(F'x_01: {x_01}')
print(F'dE_01: {deltaE_01}')

total_element = (0+1j) * X.q_au * deltaE_01 * np.sqrt(2) * X.A0_au * x_01
print(F'Expected element is {total_element}')
agrees = np.isclose(total_element, X.compute_coupling_element_p_dot_A(0, 1, 1, 2))
print(F' Agrees: {agrees}')

Recall the three coupled Hamiltonians:
p dot A Hamiltonian,
\begin{equation}
    \hat{H}_{\rm p \cdot A} = \frac{\hat{p}^2}{2m} + V(\hat{x})  + \hbar \omega \hat{b}^{\dagger} \hat{b} -  \frac{z}{m} \hat{p} \cdot  {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) + \frac{z^2}{2m}  {\bf A}_0^2 ( \hat{b}^{\dagger} + \hat{b})^2,
\end{equation}
the d dot E Hamiltonian,
\begin{equation}
    \hat{H}_{\rm d \cdot E} = \frac{\hat{p}^2}{2m} + V(\hat{x}) + \hbar \omega \hat{b}^{\dagger} \hat{b} + i\omega \hat{\mu} \cdot {\bf A}_0 ( \hat{b}^{\dagger} - \hat{b}) +\frac{\omega_{{\rm cav}}}{\hbar} ( \hat{\mu} \cdot {\bf A}_0)^2
\end{equation}
or the Pauli-Fierz (PF) Hamiltonian,
\begin{equation}
    \hat{H}_{\rm d \cdot E} = \frac{\hat{p}^2}{2m}+ V(\hat{x}) + \hbar \omega \hat{b}^{\dagger} \hat{b} - \omega \hat{\mu} \cdot {\bf A}_0 ( \hat{b}^{\dagger} + \hat{b}) +\frac{\omega_{{\rm cav}}}{\hbar} ( \hat{\mu} \cdot {\bf A}_0)^2
\end{equation}

The matter Hamiltonian terms $\frac{\hat{p}^2}{2m} + V(\hat{x})$ have been solved by either our Morse, quartic perturbation, or finite difference approach and our photon Hamiltonian term $\hbar \omega \hat{b}^{\dagger} \hat{b}$ is also already known.  We need to evaluate 
the additional coupling elements.  For the d.E and PF Hamiltonians, the coupling elements will involve matter matrix elements with of the form 
\begin{equation}
\langle \psi_i | \hat{\mu} | \psi_f \rangle = z \langle \psi_i | \hat{x} | \psi_f \rangle
\end{equation}
and the p.A Hamiltonian will have matrix elements of the form
\begin{equation}
\langle \psi_i | \hat{p} | \psi_f \rangle = i m \left(E_i - E_f \right) \langle \psi_i | \hat{x} | \psi_f \rangle
\end{equation}
where $E_n$ is the energy associated with matter state $n$, $m$ is the reduced mass, and $i$ is the imaginary unit.  Hence, we need to be able to compute matrix elements of the position operator with our matter states.  The following function will accomplish this.


In [ ]:
X.photon_dim = 3
X.matter_dim = 3
X.build_basis()
print(X.basis)
print(len(X.basis))

In [ ]:
print(X.basis[3][0])

In [ ]:
X.build_p_dot_A_Hamiltonian()

print(X.H_p_dot_A)
H_pda = X.H_p_dot_A
print(np.allclose(H_pda, H_pda.conj().T))

In [ ]:
from numpy import linalg as la
X.build_d_dot_E_Hamiltonian()
X.build_PF_Hamiltonian()

vals_PdA, vecs_PdA = la.eigh(X.H_p_dot_A)
vals_DdE, vecs_EdE = la.eigh(X.H_d_dot_E)
vals_PF, vecs_PF = la.eigh(X.H_PF)

In [ ]:
print(vals_DdE[2]-vals_DdE[1])
print(vals_PdA[2]-vals_PdA[1])
print(vals_PF[2]-vals_PdA[1])

In [ ]:
print(vals_DdE[0])
print(vals_PdA[0])
print(vals_PF[0])

In [ ]:

A_vals = [0, 0.002, 0.004, 0.006, 0.008, 0.010, 0.012, 0.014, 0.016, 0.018, 0.02]
from matplotlib import pyplot as plt

pda_vals = np.zeros((11,4))
dde_vals = np.zeros((11,4))
pf_vals = np.zeros((11,4))

idx = 0
for Av in A_vals:
    X.A0_au = Av
    X.build_d_dot_E_Hamiltonian()
    X.build_p_dot_A_Hamiltonian()
    X.build_PF_Hamiltonian()
    vals_PdA, vecs_PdA = la.eigh(X.H_p_dot_A)
    vals_DdE, vecs_EdE = la.eigh(X.H_d_dot_E)
    vals_PF, vecs_PF = la.eigh(X.H_PF)
    pda_vals[idx,:] = vals_PdA[:4]
    dde_vals[idx,:] = vals_DdE[:4]
    pf_vals[idx,:] = vals_PF[:4]
    idx += 1

    
    


In [ ]:
#plt.plot(A_vals, pda_vals[:,0])
#plt.plot(A_vals, dde_vals[:,0])
plt.plot(A_vals, pf_vals[:,1])
#plt.plot(A_vals, pf_vals[:,2])

plt.show()
